Prueba con 2 tipos de condiciones: ADHD y Anxiety

# Imports

In [ ]:
%pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.2 MB/s eta 0:00:00


In [ ]:
import torch
import random
import warnings
import numpy as np
import pandas as pd
import seaborn as sn
import tensorflow as tf
from tqdm import trange
from tabulate import tabulate
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

#GPU

Importante activar el tipo de entorno GPU si se ejecuta en colab

In [ ]:
!nvidia-smi

Tue May  9 12:47:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
torch.cuda.is_available()

True

In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

# Lectura de los datos

## Carga de datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


*

In [ ]:
%cd ./drive/Shareddrives/DL + NLP/Proyecto DL + NLP/dataset/RedditMHD/Data

[Errno 2] No such file or directory: './drive/MyDrive/2022-23/Master/Proyecto DL + NLP/dataset/RedditMHD/Data'
/content


In [ ]:
%ls

ADHD/  Anxiety/  Bipolar/  BPD/  Depression/  EDA/  PTSD/  Schizophrenia/


In [ ]:
%cd ADHD

/content/drive/.shortcut-targets-by-id/1j0RASnE3IntTbVJIRuwXAvt885-X9BEv/Proyecto DL + NLP/dataset/RedditMHD/Data/ADHD


In [ ]:
path = 'adhd_2019_features_tfidf_256.csv'

In [ ]:
df_adhd = pd.read_csv(path)

In [ ]:
df_adhd.head()

,subreddit,author,date,post,automated_readability_index,coleman_liau_index,flesch_kincaid_grade_level,flesch_reading_ease,gulpease_index,gunning_fog_index,...,tfidf_wish,tfidf_without,tfidf_wonder,tfidf_work,tfidf_worri,tfidf_wors,tfidf_would,tfidf_wrong,tfidf_x200b,tfidf_year
0,adhd,slpgh,2019/01/01,Has anyone tried the GeneSight genetic test fo...,7.221590,7.911672,7.542360,70.248696,63.869565,11.267081,...,0.0,0.000000,0.106994,0.143653,0.0,0.000000,0.000000,0.0,0.0,0.0
1,adhd,-Warrior_Princess-,2019/01/01,I was doing so well... I was diagnosed back in...,3.425192,5.891336,4.237154,83.207846,76.115385,7.621538,...,0.0,0.000000,0.000000,0.173533,0.0,0.000000,0.000000,0.0,0.0,0.0
2,adhd,VeryOriginalName98,2019/01/01,Is it possible to have ADHD and not typically ...,5.034505,7.626343,6.001868,71.878846,71.564103,10.610989,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
3,adhd,sleepy_sloth_1,2019/01/01,"Is it possible to treat ADD, anxiety, and depr...",1.902496,3.581086,3.640000,89.425513,77.772321,6.291300,...,0.0,0.196092,0.000000,0.000000,0.0,0.069101,0.048168,0.0,0.0,0.0
4,adhd,TheBulgarianBrute,2019/01/01,Everything seems very complicated. I was diagn...,5.888431,6.327416,6.313906,78.778517,66.709497,9.637583,...,0.0,0.000000,0.000000,0.179662,0.0,0.000000,0.000000,0.0,0.0,0.0


In [ ]:
%cd ..

/content/drive/.shortcut-targets-by-id/1j0RASnE3IntTbVJIRuwXAvt885-X9BEv/Proyecto DL + NLP/dataset/RedditMHD/Data


In [ ]:
%ls

ADHD/  Anxiety/  Bipolar/  BPD/  Depression/  EDA/  PTSD/  Schizophrenia/


In [ ]:
%cd Anxiety

/content/drive/.shortcut-targets-by-id/1j0RASnE3IntTbVJIRuwXAvt885-X9BEv/Proyecto DL + NLP/dataset/RedditMHD/Data/Anxiety


In [ ]:
path = 'anxiety_2019_features_tfidf_256.csv'

In [ ]:
df_anxiety = pd.read_csv(path)

In [ ]:
df_anxiety.head()

,subreddit,author,date,post,automated_readability_index,coleman_liau_index,flesch_kincaid_grade_level,flesch_reading_ease,gulpease_index,gunning_fog_index,...,tfidf_wish,tfidf_without,tfidf_wonder,tfidf_work,tfidf_worri,tfidf_wors,tfidf_would,tfidf_wrong,tfidf_x200b,tfidf_year
0,anxiety,SkyRyDyMama,2019/01/01,Extreme anxiety after Road Rage incident Almos...,1.129031,3.009227,3.170567,90.933889,80.914894,5.028842,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
1,anxiety,14Smith15,2019/01/01,Can’t sleep because I can’t breathe I am curre...,1.593536,3.218761,3.174643,92.657500,78.500000,5.571429,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
2,anxiety,Throwawayer98,2019/01/01,Is this anxiety? I need help. Hello.\n\n Not t...,1.950219,3.897757,3.896364,86.401745,78.570957,6.440783,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.057080,0.087852,0.0,0.000000
3,anxiety,spirituous-crab,2019/01/01,"DAE get really anxious on ""lazy days""? I just ...",0.956538,3.025588,2.130449,97.154135,82.589744,5.438462,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.132203
4,anxiety,rosehnova,2019/01/01,Best way to deal with new job anxiety? I have ...,5.300862,5.871315,6.504330,76.441760,68.017341,9.990331,...,0.0,0.0,0.0,0.186269,0.129919,0.0,0.090878,0.000000,0.0,0.082383


## Información de los datos

### ADHD

In [ ]:
df_adhd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10223 entries, 0 to 10222
Columns: 350 entries, subreddit to tfidf_year
dtypes: float64(270), int64(76), object(4)
memory usage: 27.3+ MB


In [ ]:
df_adhd['subreddit'].value_counts() 

adhd    10223
Name: subreddit, dtype: int64

In [ ]:
df_adhd['author'].value_counts() 

slpgh              1
h_anorak           1
JoshsFace          1
memekitten69       1
zenhangeki         1
                  ..
throwaway_20_20    1
__us3r1999         1
DondoMidas         1
indyur1            1
Rue_Anemone        1
Name: author, Length: 10223, dtype: int64

In [ ]:
df_adhd['date'].value_counts() 

2019/01/09    140
2019/02/06    133
2019/01/29    125
2019/02/20    122
2019/01/02    120
             ... 
2019/02/23     60
2019/03/23     60
2019/04/13     59
2019/03/09     59
2019/03/16     52
Name: date, Length: 110, dtype: int64

### Anxiety

In [ ]:
df_anxiety.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13213 entries, 0 to 13212
Columns: 350 entries, subreddit to tfidf_year
dtypes: float64(270), int64(76), object(4)
memory usage: 35.3+ MB


In [ ]:
df_anxiety['subreddit'].value_counts() 

anxiety    13213
Name: subreddit, dtype: int64

In [ ]:
df_anxiety['author'].value_counts() 

SkyRyDyMama             1
maoishere               1
TheyreMineralsMarie1    1
proflayton9999          1
SlippyTheFeeler         1
                       ..
geepscreeps             1
Cattlerancher7000       1
Samdolph                1
takeawayaddicted        1
palfor3                 1
Name: author, Length: 13213, dtype: int64

In [ ]:
df_anxiety['date'].value_counts() 

2019/01/02    199
2019/01/03    171
2019/01/07    170
2019/01/16    165
2019/01/06    160
             ... 
2019/03/22     90
2019/04/02     88
2019/03/30     86
2019/04/20     81
2019/04/12     77
Name: date, Length: 110, dtype: int64

# Tratamiento de los datos

### ADHD

In [ ]:
df_adhd['type'] = 0

In [ ]:
df_adhd = df_adhd.filter(['post', 'type'], axis = 1)

In [ ]:
df.rename(columns={"post": "text", "type": "labels"})

In [ ]:
df_adhd.head()

,post,type
0,Has anyone tried the GeneSight genetic test fo...,0
1,I was doing so well... I was diagnosed back in...,0
2,Is it possible to have ADHD and not typically ...,0
3,"Is it possible to treat ADD, anxiety, and depr...",0
4,Everything seems very complicated. I was diagn...,0


In [ ]:
df_adhd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10223 entries, 0 to 10222
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   post    10223 non-null  object
 1   type    10223 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 159.9+ KB


### Anxiety

In [ ]:
df_anxiety['type'] = 1

In [ ]:
df_anxiety = df_anxiety.filter(['post', 'type'], axis = 1)

In [ ]:
df_anxiety.head()

,post,type
0,Extreme anxiety after Road Rage incident Almos...,1
1,Can’t sleep because I can’t breathe I am curre...,1
2,Is this anxiety? I need help. Hello.\n\n Not t...,1
3,"DAE get really anxious on ""lazy days""? I just ...",1
4,Best way to deal with new job anxiety? I have ...,1


In [ ]:
df_anxiety.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13213 entries, 0 to 13212
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   post    13213 non-null  object
 1   type    13213 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 206.6+ KB


### Combinación

In [ ]:
frames = [df_adhd, df_anxiety]

result = pd.concat(frames)

In [ ]:
result = shuffle(result)

In [ ]:
result.head()

,post,type
5832,I’m starting a new job today and I’m freaking ...,1
13131,Does anybody work an at-home job? I’ve struggl...,1
9148,How do you manage this symptom? \n\nBetween w...,0
743,My half sister have her new boyfriend over and...,1
9696,Anyone here from New Zealand that's been diagn...,0


In [ ]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23436 entries, 5832 to 2593
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   post    23436 non-null  object
 1   type    23436 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 549.3+ KB


In [ ]:
result['type'].value_counts() 

1    13213
0    10223
Name: type, dtype: int64

In [ ]:
text = result['post'].values
labels = result['type'].values

# Modelo

### Preprocessing

In [ ]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
    )

In [ ]:
def print_rand_sentence():
  '''Displays the tokens and respective IDs of a random text sample'''
  index = random.randint(0, len(text)-1)
  table = np.array([tokenizer.tokenize(text[index]), 
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[index]))]).T
  print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence()

╒══════════════╤═════════════╕
│ Tokens       │   Token IDs │
╞══════════════╪═════════════╡
│ the          │        1996 │
├──────────────┼─────────────┤
│ daily        │        3679 │
├──────────────┼─────────────┤
│ struggles    │       11785 │
├──────────────┼─────────────┤
│ .            │        1012 │
├──────────────┼─────────────┤
│ .            │        1012 │
├──────────────┼─────────────┤
│ .            │        1012 │
├──────────────┼─────────────┤
│ .            │        1012 │
├──────────────┼─────────────┤
│ .            │        1012 │
├──────────────┼─────────────┤
│ .            │        1012 │
├──────────────┼─────────────┤
│ yes          │        2748 │
├──────────────┼─────────────┤
│ boss         │        5795 │
├──────────────┼─────────────┤
│ i            │        1045 │
├──────────────┼─────────────┤
│ am           │        2572 │
├──────────────┼─────────────┤
│ asking       │        4851 │
├──────────────┼─────────────┤
│ you          │        2017 │
├───────

In [ ]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 32,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )


for sample in text:
  encoding_dict = preprocessing(sample, tokenizer)
  token_id.append(encoding_dict['input_ids']) 
  attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
token_id[6]

tensor([  101,  6881,  5622, 17062,  2080,  2076,  3949,  2007,  1062, 12898,
         6199,  1006, 14262, 21493,  3170,  1007,  7592,  4364,  1012,  1045,
         9015,  2013, 10089,  2004,  2116,  2500,  2182,  1012,  2009,  1521,
         1055,   102])

In [ ]:
def print_rand_sentence_encoding():
  '''Displays tokens, token IDs and attention mask of a random text sample'''
  index = random.randint(0, len(text) - 1)
  tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
  token_ids = [i.numpy() for i in token_id[index]]
  attention = [i.numpy() for i in attention_masks[index]]

  table = np.array([tokens, token_ids, attention]).T
  print(tabulate(table, 
                 headers = ['Tokens', 'Token IDs', 'Attention Mask'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence_encoding()

╒═══════════╤═════════════╤══════════════════╕
│ Tokens    │   Token IDs │   Attention Mask │
╞═══════════╪═════════════╪══════════════════╡
│ [CLS]     │         101 │                1 │
├───────────┼─────────────┼──────────────────┤
│ anyway    │        4312 │                1 │
├───────────┼─────────────┼──────────────────┤
│ to        │        2000 │                1 │
├───────────┼─────────────┼──────────────────┤
│ calm      │        5475 │                1 │
├───────────┼─────────────┼──────────────────┤
│ an        │        2019 │                1 │
├───────────┼─────────────┼──────────────────┤
│ extremely │        5186 │                1 │
├───────────┼─────────────┼──────────────────┤
│ over      │        2058 │                1 │
├───────────┼─────────────┼──────────────────┤
│ ##active  │       19620 │                1 │
├───────────┼─────────────┼──────────────────┤
│ ad        │        4748 │                1 │
├───────────┼─────────────┼──────────────────┤
│ ##hd      │

### Data split

In [ ]:
val_ratio = 0.2
batch_size = 16

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size = val_ratio,
    shuffle = True,
    stratify = labels)

# Train and validation sets
train_set = TensorDataset(token_id[train_idx], 
                          attention_masks[train_idx], 
                          labels[train_idx])

val_set = TensorDataset(token_id[val_idx], 
                        attention_masks[val_idx], 
                        labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size
        )

### Train

In [ ]:
def b_tp(preds, labels):
  '''Returns True Positives (TP): count of correct predictions of actual class 1'''
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
  '''Returns False Positives (FP): count of wrong predictions of actual class 1'''
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
  '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
  '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
  '''
  Returns the following metrics:
    - accuracy    = (TP + TN) / N
    - precision   = TP / (TP + FP)
    - recall      = TP / (TP + FN)
    - specificity = TN / (TN + FP)
  '''
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  tp = b_tp(preds, labels)
  tn = b_tn(preds, labels)
  fp = b_fp(preds, labels)
  fn = b_fn(preds, labels)
  b_accuracy = (tp + tn) / len(labels)
  b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
  b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
  b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
  return b_accuracy, b_precision, b_recall, b_specificity

In [ ]:
# Load the BertForSequenceClassification model
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(model.parameters(), 
                              lr = 5e-5,
                              eps = 1e-08
                              )

# Run on GPU
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Recommended number of epochs: 2, 3, 4. See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 2

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model to training mode
    model.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')


Epoch:  50%|█████     | 1/2 [02:20<02:20, 140.90s/it]


	 - Train loss: 0.2671
	 - Validation Accuracy: 0.9010
	 - Validation Precision: 0.9140
	 - Validation Recall: 0.9090
	 - Validation Specificity: 0.8903



Epoch: 100%|██████████| 2/2 [04:41<00:00, 140.93s/it]


	 - Train loss: 0.1694
	 - Validation Accuracy: 0.9006
	 - Validation Precision: 0.8932
	 - Validation Recall: 0.9338
	 - Validation Specificity: 0.8549



### Predict

In [ ]:
new_sentence = 'What my eyes see are too distracting for me to pay attention to what I am hearing'

# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
  output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Anxiety' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'ADHD'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

Input Sentence:  What my eyes see are too distracting for me to pay attention to what I am hearing
Predicted Class:  ADHD


In [ ]:
new_sentence = 'Of course, HR people want someone who is good at working together in a team and who can function under a lot of stress, but I\'m the opposite of that'

# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
  output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Anxiety' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'ADHD'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

Input Sentence:  Of course, HR people want someone who is good at working together in a team and who can function under a lot of stress, but I'm the opposite of that
Predicted Class:  Anxiety
